## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column and the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into the metric system.
 
*Store the result in a variable called `data`*!

In [1]:
import geopandas as gpd

data = gpd.read_file('data/Kruger_posts.shp')
print(f'Data CRS: {data.crs}')

Data CRS: epsg:4326


- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [2]:
data.to_crs(epsg=32735, inplace=True)

In [3]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                         geometry  
0  POINT (952912.890 7229683.258)  
1  POINT (953433.223 7172080.632)  
2  POINT (898955.144 7302197.408)  
3  POINT (956927.218 7243564.942)  
4  POINT (956794.955 7236187.926)  


In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [6]:
grouped = data.groupby('userid')

In [7]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [8]:
from shapely.geometry import LineString

In [29]:
user_points = []
for key, group in grouped:
    if len(group) < 2:
        user_points.append({'userid': key, \
                            'geometry': group['geometry'].iloc[0]})
    else:
        user_points.append({'userid': key, \
                            'geometry': LineString(list(group.sort_values('timestamp')['geometry']))})
movements = gpd.GeoDataFrame(user_points)

In [30]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'geopandas.geodataframe.GeoDataFrame'>
None
0    LINESTRING (942231.630 7254606.868, 938934.725...
1                       POINT (900800.126 7174374.539)
2                       POINT (975417.175 7171978.245)
3                       POINT (911006.669 7194909.337)
4    LINESTRING (905394.500 7193375.148, 905394.500...
Name: geometry, dtype: geometry


**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lengths of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [31]:
movements.set_crs(epsg=32735, inplace=True)
movements['distance'] = movements['geometry'].map(lambda entry: entry.length if type(entry) != 'Point' else 0)

In [32]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,userid,geometry,distance
0,16301,"LINESTRING (942231.630 7254606.868, 938934.725...",328455.11543
1,26589,POINT (900800.126 7174374.539),0.00000
2,29322,POINT (975417.175 7171978.245),0.00000
3,42181,POINT (911006.669 7194909.337),0.00000
4,45136,"LINESTRING (905394.500 7193375.148, 905394.500...",0.00000


You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

In [35]:
print(f"Shortest distance (including people who didn't move): {min(movements['distance'])}")
print(f"Shortest distance (including only people who moved): {min(movements[movements['distance'] > 0]['distance'])}")
print()

print(f"Mean distance (including people who didn't move): {movements['distance'].mean()}")
print(f"Mean distance (including only people who moved): {movements[movements['distance'] > 0]['distance'].mean()}")
print()

print(f"Longest distance (including people who didn't move): {max(movements['distance'])}")
print(f"Longest distance (including only people who moved): {max(movements[movements['distance'] > 0]['distance'])}")

Shortest distance (including people who didn't move): 0.0
Shortest distance (including only people who moved): 0.00010161846437100987

Mean distance (including people who didn't move): 64508.8111793036
Mean distance (including only people who moved): 117438.31425525392

Longest distance (including people who didn't move): 6970668.816343964
Longest distance (including only people who moved): 6970668.816343964


- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [37]:
# only saving the people who moved because geopandas seems to yell when the geometries in the
# geometry column are of different types

fp = 'data/some_movements.shp'
movements[movements['distance'] > 0].to_file(fp)

In [38]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!